# Llama 2 7b Chat 모델 No-tuned

In [ ]:
# 'input_text': '원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고, 노조조합장이 사임한 경우, 노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고 종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고 고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면, 이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.'
# , 'target_text': '원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.'}

## 1. 모델 및 토크나이저 로드

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_llama2_7b_chat_model(model_name="meta-llama/Llama-2-7b-chat-hf"):
    """
    Hugging Face에 공개된 Llama 2 7B Chat 모델과 토크나이저를 로드합니다.
    파인튜닝 없이, 사전학습된 모델만 사용합니다.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map = {"": "cuda:1"}
    )
    return model, tokenizer

# 사용 예시:
# model, tokenizer = load_llama2_7b_chat_model("meta-llama/Llama-2-7b-chat-hf")

## 2. 결과 도출

In [2]:
import torch

def generate_summary(model, tokenizer, input_text, max_new_tokens=150):
    """
    Llama 2 7B Chat 모델을 사용하여 요약을 생성합니다.
    (파인튜닝 없이, 사전학습된 모델만 활용)
    
    예시 프롬프트를 구성하여 모델에게 '원문'에 대한 요약을 요청합니다.
    """
    # 간단한 요약 프롬프트 예시
    prompt = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n"
        f"Please summarize the following text in Korean, using a concise and clear style.\n"
        f"### Input:\n{input_text}\n"
        f"### Response:"
    )
    
    # 모델이 사용할 디바이스 결정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 프롬프트 토큰화
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 모델 출력 디코딩
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 혹시 필요하다면, 모델이 출력한 전체 텍스트 중 불필요한 부분(Instruction, Input 등)을 제거할 수 있습니다.
    # 여기서는 간단히 요약 부분만 그대로 반환합니다.
    return summary

# 사용 예시:
# text_to_summarize = "여기에 원문 텍스트를 입력하세요."
# summary = generate_summary(model, tokenizer, text_to_summarize)
# print(summary)


## 3. 메인 함수: 전체 파이프라인 실행

In [3]:
def main():
    import os
    # GPU 메모리 단편화 완화를 위해 환경 변수 설정 (옵션)
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    
    # 1. 모델 및 토크나이저 로드
    model, tokenizer = load_llama2_7b_chat_model("meta-llama/Llama-2-7b-chat-hf")
    
    # 2. 요약할 텍스트 (예시)
    input_text = (
        "한국의 경제는 글로벌 경기 둔화와 내수 부진의 영향을 받고 있습니다. "
        "정부는 이를 극복하기 위해 다양한 정책을 발표하고, 기업들은 혁신을 통해 경쟁력을 강화하고 있습니다."
    )
    
    # 3. 결과 도출
    summary = generate_summary(model, tokenizer, input_text)
    
    print("원문:")
    print(input_text)
    print("\n생성된 요약:")
    print(summary)

if __name__ == "__main__":
    main()


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
403 Client Error. (Request ID: Root=1-67c2d905-5262feee7b5c857c23d15a65;db466264-03c4-422e-a524-1592ad74660a)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-2-7b-chat-hf to ask for access.

## 4. 성능 평가

### 1. 평가 데이터셋 로드

In [4]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from bert_score import score as bertscore

nltk.download('wordnet')
nltk.download('omw-1.4')

def evaluate_summary(predictions, references):
    """
    predictions: 모델이 생성한 요약 리스트
    references:  참조 요약 리스트 (실제 요약)
    각 항목별로 짝을 맞춰 평가
    """
    # ROUGE
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1, rouge2, rougeL = 0, 0, 0
    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        rouge1 += scores['rouge1'].fmeasure
        rouge2 += scores['rouge2'].fmeasure
        rougeL += scores['rougeL'].fmeasure
    n = len(predictions)
    rouge_scores = {
        "rouge1": rouge1 / n,
        "rouge2": rouge2 / n,
        "rougeL": rougeL / n
    }
    
    # BLEU
    refs_list = [[ref.split()] for ref in references]
    preds_list = [pred.split() for pred in predictions]
    bleu = corpus_bleu(refs_list, preds_list)
    
    # METEOR
    total_meteor = 0
    for pred, ref in zip(predictions, references):
        total_meteor += meteor_score([ref], pred)
    meteor_avg = total_meteor / n
    
    # BERTScore (한국어)
    P, R, F1 = bertscore(predictions, references, lang="ko")
    bert_f1 = F1.mean().item()
    
    results = {
        "ROUGE": rouge_scores,
        "BLEU": bleu,
        "METEOR": meteor_avg,
        "BERTScore_F1": bert_f1
    }
    return results

# 사용 예시:
# predictions = ["모델이 생성한 요약1", "모델이 생성한 요약2", ...]
# references = ["실제 요약1", "실제 요약2", ...]
# eval_results = evaluate_summary(predictions, references)
# print(eval_results)


[nltk_data] Downloading package wordnet to /home/wanted-1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/wanted-1/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
